In [1]:
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd

import matplotlib.pyplot as plt

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix

import prepare_readme as p
import explore as e

In [2]:
df = p.prep_readme()

In [3]:
df.head()

,language,readme_contents,repo,readme_contents_stemmed,readme_contents_lemmatized
0,CSS,# Welcome\n\n> **Warning**: this book is **not...,/nakov/Practical-Cryptography-for-Developers-Book,welcom warn book finish still work chapter com...,welcome warning book finished still working ch...
1,Java,# Monorepo of Deeplearning4j\n\nWelcome to the...,/eclipse/deeplearning4j,monorepo deeplearn j welcom new monorepo deepl...,monorepo deeplearning j welcome new monorepo d...
2,Java,# H2O\n\n[![Join the chat at https://gitter.im...,/h2oai/h2o-3,h join chat http gitter im h oai h http badg g...,h join chat http gitter im h oai h http badge ...
3,C++,"<div align=""center"">\n <img src=""https://www....",/tensorflow/tensorflow,div align center img src http www tensorflow o...,div align center img src http www tensorflow o...
4,C++,gRPC - An RPC library and framework\n=========...,/grpc/grpc,grpc rpc librari framework grpc modern open so...,grpc rpc library framework grpc modern open so...


In [4]:
df_v = df.language.value_counts(dropna=False)

In [5]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels

,n,percent
HTML,20,0.190476
C++,20,0.190476
Python,15,0.142857
JavaScript,10,0.095238
C,10,0.095238
Java,10,0.095238
CSS,10,0.095238
Shell,10,0.095238


In [6]:
# words that are programming languages only
#verterize ngrams

In [7]:
# restrict df to target and predicted veriables
df = df[['readme_contents_stemmed','language']]

In [8]:
# vectorize stemmed readme contents and assign X any y veriables
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(get_ngram_strings(get_ngrams(n=2))
y = df.language

SyntaxError: invalid syntax (<ipython-input-8-ca1b664d344a>, line 4)

In [ ]:
# split data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=.2)

In [ ]:
# create classifier object and fit it to the data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')

In [ ]:
def get_words(text):
    
    words = re.sub(r'[^\w\s]', '', text).split()
    return words

In [ ]:
def get_ngrams(n=2):

    ngram_master_list = []

    for row in df.readme_contents_stemmed.apply(get_words):

        bigrams = nltk.ngrams(row, n)

        ngram_master_list.extend(bigrams)
    
    return ngram_master_list

In [ ]:
def get_ngram_strings(ngrams):
    
    string_list = []

    for pair in ngrams:
    
        first = pair[0]

        second = pair[1]
    
        string_list.append(f"{first} {second}")
        
    return string_list

In [ ]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(get_ngram_strings(get_ngrams(n=2)))
y = df.language

In [ ]:
HTML_list =[]

for lst in df.readme_contents_stemmed.apply(p.tokenize)[df.language=='HTML']:
    
    HTML_list.extend(lst)
    
print(HTML_list)

In [ ]:
def tokenize(article):
    '''
    tokenizes words in a string
    '''

    # create token object
    tokenizer = nltk.tokenize.ToktokTokenizer()

    # use object to tokenize string
    article = tokenizer.tokenize(article, return_str=True)
    
    return article

#new svc or decition tree

In [ ]:
# create classifier object and fit it to the data
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
clf.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=123,
            splitter='best')